https://gis.stackexchange.com/questions/461143/convert-wrf-netcdf-to-tiff

In [421]:
import os
import netCDF4 as nc
import numpy as np
import rasterio
from rasterio.transform import from_origin
import wrf
import xarray as xr
import numpy as np
import math
import cartopy.crs as crs
import cartopy.crs as ccrs

In [422]:
folder = 'wrfout'
for filename in os.listdir(folder):
    # Check if the filename contains a colon
    if '%' in filename or ':' in filename:
        new_filename = filename.replace("%", "-")
        new_filename = filename.replace(":", "-")
        old_path = os.path.join(folder, filename)
        new_path = os.path.join(folder, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")

In [423]:
root = 'wrfout'
data = nc.Dataset(root + '/wrfout_d03_2025-03-04_08-3A00-3A00')

### Temperature

In [424]:
t2 =  wrf.getvar(data, "T2", timeidx=0)
t2

<xarray.DataArray 'T2' (south_north: 100, west_east: 100)> Size: 40kB
array([[279.6635 , 279.5325 , 279.35266, ..., 288.20364, 288.25037,
        288.29462],
       [279.42566, 279.31183, 279.15912, ..., 288.17953, 288.22162,
        288.26483],
       [279.16684, 279.06964, 278.9292 , ..., 288.13995, 288.19287,
        288.23425],
       ...,
       [286.93857, 287.01212, 287.03683, ..., 290.30173, 290.30606,
        290.30823],
       [287.17984, 287.20514, 287.37054, ..., 290.30222, 290.30698,
        290.30927],
       [287.31924, 287.42252, 287.52487, ..., 290.3012 , 290.30576,
        290.3093 ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -15.62 -15.62 ... -15.39
    XLAT     (south_north, west_east) float32 40kB 27.95 27.95 ... 28.15 28.15
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  TEMP at 2 M
    units:        K
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

### Wind

In [457]:
u10 =  wrf.getvar(data, "U10", timeidx=0)
u10

<xarray.DataArray 'U10' (south_north: 100, west_east: 100)> Size: 40kB
array([[-0.6221364 , -0.9009852 , -1.148355  , ...,  3.2323072 ,
         3.2294953 ,  3.235849  ],
       [-0.6898061 , -0.9847839 , -1.2177311 , ...,  3.3379526 ,
         3.3577263 ,  3.3137834 ],
       [-0.7303791 , -1.0518472 , -1.3288326 , ...,  3.42088   ,
         3.4582808 ,  3.40078   ],
       ...,
       [ 0.54648036,  0.5577025 ,  0.61668396, ...,  1.6824307 ,
         1.6603049 ,  1.643347  ],
       [ 0.5213582 ,  0.55247366,  0.54825896, ...,  1.6151892 ,
         1.5878958 ,  1.5785753 ],
       [ 0.47533703,  0.48875517,  0.5089321 , ...,  1.5479133 ,
         1.5380083 ,  1.5345001 ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -15.62 -15.62 ... -15.39
    XLAT     (south_north, west_east) float32 40kB 27.95 27.95 ... 28.15 28.15
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  U at 10 M
    units:        m s-1
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

In [443]:
#https://confluence.ecmwf.int/pages/viewpage.action?pageId=133262398
#https://mst.nerc.ac.uk/wind_vect_convs.html
#! The wind dirección is in convention of meteorological wind direction, which is the direction wind is coming from.

def wind_speed_direction(u10, v10):
    theta_degree = theta_degree = (180 + 180/np.pi * np.arctan2(u10, v10)) % 360
    theta_radians = np.radians(theta_degree)
    v_mod = np.sqrt(u10**2 + v10**2)
    u_vel = v_mod * np.cos(theta_radians)
    v_vel = v_mod * np.sin(theta_radians)
    return(theta_degree, v_mod, u_vel, v_vel)

In [444]:
u10 = wrf.getvar(data, 'U10', timeidx=0)  
v10 = wrf.getvar(data, 'V10', timeidx=0)

nx = data.dimensions['west_east'].size
ny = data.dimensions['south_north'].size
dt, dx, dy = data.DT, data.DX, data.DY
cen_lat, cen_lon = data.CEN_LAT, data.CEN_LON  # center of the domain
truelat1, truelat2 = data.TRUELAT1, data.TRUELAT2  # true latitudes
pole_lat, pole_lon = data.POLE_LAT, data.POLE_LON  # coordinates of the pole    

print(nx, ny, cen_lat, cen_lon)

100 100 28.049374 -15.504883


In [445]:
theta_degree, v_mod, u_vel, v_vel = wind_speed_direction(u10, v10)

### Wind by height

In [490]:
wspd =  wrf.getvar(data, "wspd", timeidx=0)
z =  wrf.getvar(data, "z", timeidx=0) #model height - [MSL] (mass grid)
hgt =  wrf.getvar(data, "HGT", timeidx=0) #Terrain Height
z_agl = z - hgt #altura sobre el terreno
z_agl

<xarray.DataArray (bottom_top: 44, south_north: 100, west_east: 100)> Size: 2MB
array([[[   24.107178,    24.091553,    24.069458, ...,    25.156158,
            25.161812,    25.167496],
        [   24.078491,    24.063843,    24.0448  , ...,    25.151604,
            25.158833,    25.163681],
        [   24.048218,    24.035156,    24.016724, ...,    25.146553,
            25.154663,    25.160004],
        ...,
        [   25.123062,    25.13266 ,    25.142303, ...,    25.313702,
            25.313725,    25.313372],
        [   25.151672,    25.162727,    25.174057, ...,    25.313938,
            25.314096,    25.313622],
        [   25.179565,    25.191124,    25.203072, ...,    25.313635,
            25.313816,    25.313818]],

       [[   74.58984 ,    74.538086,    74.46643 , ...,    77.97074 ,
            77.99263 ,    78.01368 ],
        [   74.49963 ,    74.45227 ,    74.38916 , ...,    77.95623 ,
            77.98129 ,    78.00192 ],
        [   74.404175,    74.361694,    74.30261 , ...,    77.94101 ,
            77.96756 ,    77.98972 ],
...
        [18974.111   , 18982.342   , 18991.832   , ..., 19132.383   ,
         19132.387   , 19132.385   ],
        [19000.275   , 19009.793   , 19020.254   , ..., 19132.398   ,
         19132.418   , 19132.402   ],
        [19025.54    , 19036.008   , 19046.941   , ..., 19132.383   ,
         19132.393   , 19132.402   ]],

       [[18830.473   , 18807.316   , 18777.82    , ..., 19998.988   ,
         20006.992   , 20014.51    ],
        [18801.615   , 18780.695   , 18753.783   , ..., 19994.855   ,
         20003.111   , 20010.941   ],
        [18771.516   , 18752.219   , 18727.701   , ..., 19990.338   ,
         19998.904   , 20007.143   ],
        ...,
        [19905.56    , 19913.807   , 19923.309   , ..., 20064.215   ,
         20064.219   , 20064.209   ],
        [19931.725   , 19941.256   , 19951.727   , ..., 20064.227   ,
         20064.246   , 20064.229   ],
        [19956.992   , 19967.467   , 19978.404   , ..., 20064.207   ,
         20064.219   , 20064.232   ]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -15.62 -15.62 ... -15.39
    XLAT     (south_north, west_east) float32 40kB 27.95 27.95 ... 28.15 28.15
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: bottom_top, south_north, west_east

In [497]:
from wrf import interplevel
wspd_100m = interplevel(wspd, z_agl, 50)
wspd_100m

<xarray.DataArray 'wspd_wdir_interp' (south_north: 100, west_east: 100)> Size: 40kB
array([[6.0971365, 6.0176682, 5.946143 , ..., 2.317361 , 2.3938913,
        2.4882765],
       [6.0236583, 5.9486785, 5.8889914, ..., 2.3351884, 2.413074 ,
        2.4995368],
       [6.084677 , 6.035492 , 5.9519744, ..., 2.350521 , 2.4026532,
        2.4932132],
       ...,
       [8.53665  , 8.554373 , 8.533091 , ..., 5.9953184, 5.956507 ,
        5.9481897],
       [8.4952755, 8.543515 , 8.554909 , ..., 6.0355153, 6.0066757,
        5.990797 ],
       [8.488584 , 8.501251 , 8.496816 , ..., 6.0802727, 6.062845 ,
        6.037374 ]], dtype=float32)
Coordinates:
    wspd_wdir  <U4 16B 'wspd'
    XLONG      (south_north, west_east) float32 40kB -15.62 -15.62 ... -15.39
    XLAT       (south_north, west_east) float32 40kB 27.95 27.95 ... 28.15 28.15
    XTIME      float32 4B 120.0
    Time       datetime64[ns] 8B 2025-03-04T08:00:00
    level      int32 4B 50
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:      104
    units:          m s-1
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     LambertConformal(stand_lon=-17.017000198364258, moad_cen_...
    missing_value:  9.969209968386869e+36
    _FillValue:     9.969209968386869e+36
    vert_units:     None

### Geopotential Height

In [ ]:
ph =  wrf.getvar(data, "PH", timeidx=0)
ph

<xarray.DataArray 'PH' (bottom_top_stag: 45, south_north: 100, west_east: 100)> Size: 2MB
array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 2.66772658e-01,  3.00986260e-01,  3.00201416e-01, ...,
          4.22461939e+00,  4.22602510e+00,  4.22336721e+00],
        [ 1.21303968e-01,  1.42335683e-01,  1.62706003e-01, ...,
          4.18620014e+00,  4.22614527e+00,  4.20375490e+00],
        [-3.45367342e-02, -2.30043125e-03, -6.74481830e-03, ...,
          4.15176582e+00,  4.19608974e+00,  4.17931271e+00],
...
          5.94671729e+03,  5.94675244e+03,  5.94669287e+03],
        [ 5.93717529e+03,  5.93741309e+03,  5.93743506e+03, ...,
          5.94652930e+03,  5.94666846e+03,  5.94661084e+03],
        [ 5.93699951e+03,  5.93701758e+03,  5.93703076e+03, ...,
          5.94625293e+03,  5.94639062e+03,  5.94653076e+03]],

       [[ 6.20165771e+03,  6.20166211e+03,  6.20166748e+03, ...,
          6.20473486e+03,  6.20474707e+03,  6.20476660e+03],
        [ 6.20153711e+03,  6.20173096e+03,  6.20176660e+03, ...,
          6.20485010e+03,  6.20483789e+03,  6.20466553e+03],
        [ 6.20141406e+03,  6.20161279e+03,  6.20169385e+03, ...,
          6.20481299e+03,  6.20475439e+03,  6.20456445e+03],
        ...,
        [ 6.18722998e+03,  6.18775146e+03,  6.18804443e+03, ...,
          6.19890283e+03,  6.19890771e+03,  6.19871533e+03],
        [ 6.18705176e+03,  6.18750342e+03,  6.18770068e+03, ...,
          6.19867920e+03,  6.19875781e+03,  6.19862646e+03],
        [ 6.18686816e+03,  6.18703320e+03,  6.18719141e+03, ...,
          6.19831445e+03,  6.19842871e+03,  6.19853369e+03]]],
      dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -15.62 -15.62 ... -15.39
    XLAT     (south_north, west_east) float32 40kB 27.95 27.95 ... 28.15 28.15
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: bottom_top_stag, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  perturbation geopotential
    units:        m2 s-2
    stagger:      Z
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

In [476]:
phb =  wrf.getvar(data, "PHB", timeidx=0)
phb

<xarray.DataArray 'PHB' (bottom_top_stag: 45, south_north: 100, west_east: 100)> Size: 2MB
array([[[ 1.2105914e+04,  1.2333072e+04,  1.2622342e+04, ...,
          6.4898273e+02,  5.7054999e+02,  4.9688702e+02],
        [ 1.2388848e+04,  1.2594180e+04,  1.2858145e+04, ...,
          6.8958673e+02,  6.0866486e+02,  5.3173236e+02],
        [ 1.2683862e+04,  1.2873335e+04,  1.3113906e+04, ...,
          7.3382355e+02,  6.4977393e+02,  5.6892200e+02],
        ...,
        [ 1.5458354e+03,  1.4653270e+03,  1.3723367e+03, ...,
          4.3856972e-01,  4.1144994e-01,  4.2308298e-01],
        [ 1.2890126e+03,  1.1958942e+03,  1.0933359e+03, ...,
          8.3004735e-02,  6.6382714e-02,  1.2069264e-01],
        [ 1.0411213e+03,  9.3844678e+02,  8.3124432e+02, ...,
          0.0000000e+00,  0.0000000e+00, -1.8272550e-08]],

       [[ 1.2578630e+04,  1.2805448e+04,  1.3094285e+04, ...,
          1.1383220e+03,  1.0599987e+03,  9.8644995e+02],
        [ 1.2861146e+04,  1.3066171e+04,  1.3329738e+04, ...,
          1.1788750e+03,  1.0980551e+03,  1.0212401e+03],
        [ 1.3155723e+04,  1.3344904e+04,  1.3585121e+04, ...,
          1.2230471e+03,  1.1391124e+03,  1.0583821e+03],
...
        [ 1.8629745e+05,  1.8629742e+05,  1.8629745e+05, ...,
          1.8629792e+05,  1.8629792e+05,  1.8629795e+05],
        [ 1.8629748e+05,  1.8629752e+05,  1.8629755e+05, ...,
          1.8629789e+05,  1.8629795e+05,  1.8629792e+05],
        [ 1.8629761e+05,  1.8629761e+05,  1.8629764e+05, ...,
          1.8629792e+05,  1.8629792e+05,  1.8629792e+05]],

       [[ 1.9521359e+05,  1.9521356e+05,  1.9521347e+05, ...,
          1.9521706e+05,  1.9521709e+05,  1.9521712e+05],
        [ 1.9521353e+05,  1.9521347e+05,  1.9521341e+05, ...,
          1.9521703e+05,  1.9521706e+05,  1.9521706e+05],
        [ 1.9521344e+05,  1.9521341e+05,  1.9521338e+05, ...,
          1.9521700e+05,  1.9521700e+05,  1.9521709e+05],
        ...,
        [ 1.9521678e+05,  1.9521675e+05,  1.9521678e+05, ...,
          1.9521725e+05,  1.9521725e+05,  1.9521728e+05],
        [ 1.9521681e+05,  1.9521684e+05,  1.9521688e+05, ...,
          1.9521722e+05,  1.9521728e+05,  1.9521725e+05],
        [ 1.9521694e+05,  1.9521694e+05,  1.9521697e+05, ...,
          1.9521725e+05,  1.9521725e+05,  1.9521725e+05]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -15.62 -15.62 ... -15.39
    XLAT     (south_north, west_east) float32 40kB 27.95 27.95 ... 28.15 28.15
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: bottom_top_stag, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  base-state geopotential
    units:        m2 s-2
    stagger:      Z
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

In [507]:
# La altura geopotencial es una medida que toma en cuenta la variación de la gravedad con la altura y la latitud
# Se puede considerar igual que la altura física para la troposfera
z_gph = (ph + phb) / 9.81
z_gph

<xarray.DataArray (bottom_top_stag: 45, south_north: 100, west_east: 100)> Size: 2MB
array([[[ 1.23403809e+03,  1.25719385e+03,  1.28668103e+03, ...,
          6.61552200e+01,  5.81600380e+01,  5.06510696e+01],
        [ 1.26287939e+03,  1.28381030e+03,  1.31071802e+03, ...,
          7.02942581e+01,  6.20453453e+01,  5.42030907e+01],
        [ 1.29295227e+03,  1.31226648e+03,  1.33678955e+03, ...,
          7.48036194e+01,  6.62358704e+01,  5.79940834e+01],
        ...,
        [ 1.57577515e+02,  1.49370743e+02,  1.39891602e+02, ...,
          4.47063930e-02,  4.19418886e-02,  4.31277230e-02],
        [ 1.31397812e+02,  1.21905617e+02,  1.11451164e+02, ...,
          8.46123695e-03,  6.76684128e-03,  1.23030208e-02],
        [ 1.06128571e+02,  9.56622543e+01,  8.47343826e+01, ...,
          0.00000000e+00,  0.00000000e+00, -1.86264515e-09]],

       [[ 1.28225244e+03,  1.30537695e+03,  1.33482007e+03, ...,
          1.16467545e+02,  1.08483658e+02,  1.00986069e+02],
        [ 1.31103638e+03,  1.33193811e+03,  1.35880737e+03, ...,
          1.20597466e+02,  1.12363014e+02,  1.04530457e+02],
        [ 1.34104871e+03,  1.36033655e+03,  1.38482300e+03, ...,
          1.25096718e+02,  1.16545204e+02,  1.08314102e+02],
...
          1.95968027e+04,  1.95968066e+04,  1.95968027e+04],
        [ 1.95957852e+04,  1.95958125e+04,  1.95958184e+04, ...,
          1.95967793e+04,  1.95968008e+04,  1.95967910e+04],
        [ 1.95957793e+04,  1.95957812e+04,  1.95957871e+04, ...,
          1.95967539e+04,  1.95967695e+04,  1.95967832e+04]],

       [[ 2.05316250e+04,  2.05316211e+04,  2.05316133e+04, ...,
          2.05322930e+04,  2.05322969e+04,  2.05323027e+04],
        [ 2.05316055e+04,  2.05316211e+04,  2.05316172e+04, ...,
          2.05323008e+04,  2.05323027e+04,  2.05322852e+04],
        [ 2.05315840e+04,  2.05316016e+04,  2.05316055e+04, ...,
          2.05322949e+04,  2.05322871e+04,  2.05322773e+04],
        ...,
        [ 2.05304805e+04,  2.05305293e+04,  2.05305625e+04, ...,
          2.05317168e+04,  2.05317168e+04,  2.05317012e+04],
        [ 2.05304648e+04,  2.05305137e+04,  2.05305371e+04, ...,
          2.05316895e+04,  2.05317051e+04,  2.05316895e+04],
        [ 2.05304590e+04,  2.05304746e+04,  2.05304941e+04, ...,
          2.05316562e+04,  2.05316680e+04,  2.05316797e+04]]],
      dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -15.62 -15.62 ... -15.39
    XLAT     (south_north, west_east) float32 40kB 27.95 27.95 ... 28.15 28.15
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: bottom_top_stag, south_north, west_east

### Height

In [509]:
# Model Height for Mass Grid (MSL, Meal Sea Level)
# Es la altura física real desde un punto de referencia (generalmente el nivel del mar). No corrige por la variación de la gravedad
hgt =  wrf.getvar(data, "HGT", timeidx=0)
hgt

<xarray.DataArray 'HGT' (south_north: 100, west_east: 100)> Size: 40kB
array([[ 1.2340381e+03,  1.2571938e+03,  1.2866810e+03, ...,
         6.6155220e+01,  5.8160034e+01,  5.0651070e+01],
       [ 1.2628794e+03,  1.2838103e+03,  1.3107180e+03, ...,
         7.0294258e+01,  6.2045345e+01,  5.4203094e+01],
       [ 1.2929523e+03,  1.3122665e+03,  1.3367896e+03, ...,
         7.4803619e+01,  6.6235870e+01,  5.7994087e+01],
       ...,
       [ 1.5757751e+02,  1.4937074e+02,  1.3989160e+02, ...,
         4.4706393e-02,  4.1941889e-02,  4.3127723e-02],
       [ 1.3139781e+02,  1.2190562e+02,  1.1145116e+02, ...,
         8.4612370e-03,  6.7668408e-03,  1.2303021e-02],
       [ 1.0612857e+02,  9.5662254e+01,  8.4734383e+01, ...,
         0.0000000e+00,  0.0000000e+00, -1.8626451e-09]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -15.62 -15.62 ... -15.39
    XLAT     (south_north, west_east) float32 40kB 27.95 27.95 ... 28.15 28.15
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Terrain Height
    units:        m
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

#### Aspect (orientación) and slope (pendiente)

In [531]:
grad_y, grad_x = np.gradient(hgt, dy, dx)

slope = np.arctan(np.sqrt(grad_x**2 + grad_y**2))
slope_deg = np.degrees(slope)

# Aspecto (orientación del terreno)
aspect = np.arctan2(-grad_y, grad_x)
aspect_deg = np.degrees(aspect)
aspect_deg = (aspect_deg + 360) % 360

In [499]:
xmin = float(np.min(t2.XLONG).values)
xmax = float(np.max(t2.XLONG).values)
ymin = float(np.min(t2.XLAT).values)
ymax = float(np.max(t2.XLAT).values)


print( xmin, xmax, ymin, ymax)

-15.61920166015625 -15.390350341796875 27.948341369628906 28.15030288696289


In [536]:
#Extracting coordinates
latitudes = data.variables["XLAT"][0,:,:]
longitudes = data.variables["XLONG"][0,:,:]

# take a single time step
time0_T2 = data.variables["T2"][0,:,:]

# # take a signgle wind step
# time0_U10 = data.variables["U10"][0,:,:]
# time0_V10 = data.variables["V10"][0,:,:]

# defining lists for the output dataset
lat, lon, temp, wind_speed_10m, wind_direction_10m, height, slope, aspect, nc = [], [], [], [], [], [], [], [], []

In [537]:
# counter variable for cell indexing
count = 1
for i in range(latitudes.shape[0]):
    for j in range(latitudes.shape[1]):        
        latitude = latitudes[i, j]
        longitude = longitudes[i, j]
        if ymin <= latitude <= ymax and xmin <= longitude <= xmax:               
            lat.append(latitude)
            lon.append(longitude)
            # convert temperature from Kelvin to Celsius
            temp.append(time0_T2[i, j])
            wind_speed_10m.append(v_mod[i, j])
            wind_direction_10m.append(theta_degree[i, j])
            height.append(hgt[i, j])
            slope.append(slope_deg[i, j])
            aspect.append(aspect_deg[i, j]) 
            nc.append(count)
            count = count + 1

In [538]:
# converting to numpy column array
A = np.transpose(np.array([nc,lon, lat, temp,wind_speed_10m, wind_direction_10m, height, slope, aspect]))

A

array([[ 1.00000000e+00, -1.56192017e+01,  2.79508476e+01, ...,
         1.23403809e+03,  9.37601471e+00,  3.08759857e+02],
       [ 2.00000000e+00, -1.56169128e+01,  2.79508286e+01, ...,
         1.25719385e+03,  9.48721123e+00,  3.14680725e+02],
       [ 3.00000000e+00, -1.56146240e+01,  2.79508095e+01, ...,
         1.28668103e+03,  1.01916456e+01,  3.23352386e+02],
       ...,
       [ 9.99800000e+03, -1.53949280e+01,  2.81478386e+01, ...,
         0.00000000e+00,  4.44240198e-02,  1.77207520e+02],
       [ 9.99900000e+03, -1.53926392e+01,  2.81478043e+01, ...,
         0.00000000e+00,  1.73085439e-03,  9.00000000e+01],
       [ 1.00000000e+04, -1.53903503e+01,  2.81477776e+01, ...,
        -1.86264515e-09,  3.14692501e-03,  9.00000000e+01]])

In [539]:
# writing data to csv
np.savetxt('time0_T2.csv',A,delimiter=",",header="cell_number, LON, LAT, T2_degC,WIND_SPEED_10m, WIND_DIRECTION_10m, HEIGHT, SLOPE, ASPECT")

In [540]:
# # x and y resolutions
deltax = (np.max(A[:,1]) - np.min(A[:,1]))/ (nx - 1)
deltay = (np.max(A[:,2]) - np.min(A[:,2]))/ (ny - 1)

print(deltax, deltay)

0.0023116294783775255 0.002040015326605903


In [541]:
# # up left corner
minx = np.min(A[:,1])
maxy = np.max(A[:,2])

print(minx, maxy)

-15.61920166015625 28.15030288696289


In [542]:
# set the transformation for rasterio. Note that the shift operated is in order to
# make the lat,lon points to be the center of each grid cell
transform = from_origin(minx-0.5*deltax,maxy+0.5*deltay, deltax, deltay)
transform

Affine(0.0023116294783775255, 0.0, -15.620357474895439,
       0.0, -0.002040015326605903, 28.151322894626194)

In [ ]:
t2 = A[:,3]
data_band = np.flipud(t2.reshape(ny, nx))
# write it to tiff format
with rasterio.open('t4.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=t2.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

CPLE_AppDefinedError: Deleting t4.tiff failed: Permission denied

In [ ]:
wind_speed = A[:,4]
data_band = np.flipud(wind_speed.reshape(ny, nx))
# write it to tiff format
with rasterio.open('v1.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=wind_speed.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [ ]:
wind_direction = A[:,5]
data_band = np.flipud(wind_direction.reshape(ny, nx))
# write it to tiff format
with rasterio.open('vd1.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=wind_direction.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [ ]:
height = A[:,3]
data_band = np.flipud(height.reshape(ny, nx))
# write it to tiff format
with rasterio.open('vd1.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=height.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [ ]:
slope = A[:,4]
data_band = np.flipud(slope.reshape(ny, nx))
# write it to tiff format
with rasterio.open('hgt.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=slope.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [ ]:
Aspect = A[:,4]
data_band = np.flipud(Aspect.reshape(ny, nx))
# write it to tiff format
with rasterio.open('hgt.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=Aspect.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))